<a href="https://colab.research.google.com/github/alexlopespereira/mba_enap/blob/main/CD/Notebooks/Aula8_ContarDosesCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contar doses de vacinas da COVID-19

In [ ]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'hardy-messenger-229417' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!



Authenticated


### Dica: Restrinja seus testes aos estados do AC, AP e RR para diminuir o tamanho da sua Query.

In [ ]:


df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,vacina
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única|^Dose$') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89') and sigla_uf in ('AC','AP','RR')
group by sigla_uf, vacina
limit 20;
''', project_id=project_id)

df_vacina.head()

/tmp/ipython-input-4248334805.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina = pd.io.gbq.read_gbq('''


,sigla_uf,vacina,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,AC,86,185531,100467,84938,109,17,0
1,AC,87,457364,243218,156853,49567,7726,0
2,AC,85,422293,222991,195394,3584,324,0
3,AC,89,25492,12102,13375,15,0,0
4,AC,88,14251,0,0,1874,150,12227


### Calcular a semana epidemiológica (domingo a sábado) a partir da data de vacinação

In [ ]:
df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,(case
        when vacina='86' then 'Coronavac'
        when vacina='87' then 'Pfizer'
        when vacina='88' then 'Janssen'
        else 'Astrazeneca'
      end
    ) as vacina_apelido
    ,(case when EXTRACT(YEAR FROM data_aplicacao) >= 2021 then DATE_TRUNC(data_aplicacao, WEEK(SUNDAY)) else PARSE_DATE('%d/%m/%Y',  '03/01/2021') end) as semana
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|^Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89') and sigla_uf in ('AC','AP','RR')
group by vacina_apelido, sigla_uf, semana
''', project_id=project_id)


df_vacina.to_gbq("enapdatasets.vacinacao",
  project_id=project_id,
  chunksize=40000,
  if_exists='replace',
  )
df_vacina.head()


/tmp/ipython-input-3709925176.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina = pd.io.gbq.read_gbq('''
/tmp/ipython-input-3709925176.py:24: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_vacina.to_gbq("enapdatasets.vacinacao",
100%|██████████| 1/1 [00:00<00:00, 16777.22it/s]


,sigla_uf,vacina_apelido,semana,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,MT,Pfizer,2021-10-03,121243,59411,48260,12010,1562,0
1,MT,Coronavac,2021-09-12,30627,15234,15391,2,0,0
2,MT,Coronavac,2021-04-18,51801,14017,37783,1,0,0
3,MT,Coronavac,2021-06-06,2352,1291,1061,0,0,0
4,AM,Pfizer,2021-08-08,98844,95326,3499,19,0,0


## Query para projeção de 2a dose da vacina do Covid
### Requisito de negócio: 2a dose deve ser aplicada a partir de 56 dias após a 1a dose.
#### Também seria útil conhecer a demanda com antecedência (21 dias depois de aplicada a 1a dose)

In [ ]:
df_vacina_proj_teste = pd.io.gbq.read_gbq('''
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL 21 DAY) as semana_proj, semana
    FROM `enapdatasets.vacinacao`vp
where sigla_uf='RR'
order by vp.sigla_uf, vp.vacina, vp.semana, vp.sigla_uf, vp.vacina_apelido
''', project_id=project_id)

df_vacina_proj_teste.head(15)

/tmp/ipython-input-2297292228.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj_teste = pd.io.gbq.read_gbq('''


,sigla_uf_proj,vacina_apelido_proj,qt_D2_Proj,semana_proj,semana
0,RR,Astrazeneca,4,2021-02-07,2021-01-17
1,RR,Astrazeneca,10,2021-02-14,2021-01-24
2,RR,Astrazeneca,357,2021-02-21,2021-01-31
3,RR,Astrazeneca,4,2021-02-28,2021-02-07
4,RR,Astrazeneca,49,2021-03-07,2021-02-14
5,RR,Astrazeneca,368,2021-03-14,2021-02-21
6,RR,Astrazeneca,2243,2021-03-21,2021-02-28
7,RR,Astrazeneca,332,2021-03-28,2021-03-07
8,RR,Astrazeneca,258,2021-04-04,2021-03-14
9,RR,Astrazeneca,432,2021-04-11,2021-03-21


In [ ]:
df_vacina_proj = pd.io.gbq.read_gbq('''
SELECT v.sigla_uf, v.vacina_apelido, v.semana, v.qt_total, v.qt_D1, v.qt_D2, v.qt_Reforco, v.qt_Adicional, v.qt_Unica, vp.qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp. vacina_apelido_proj
FROM `enapdatasets.vacinacao` v
LEFT JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL 21 DAY) as semana_proj
    FROM `enapdatasets.vacinacao`
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=v.semana
where sigla_uf in ('AC','AP','RR')
order by v.sigla_uf, v.semana, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
''', project_id=project_id)

df_vacina_proj.head()

/tmp/ipython-input-1813122139.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj = pd.io.gbq.read_gbq('''


,sigla_uf,vacina_apelido,semana,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
0,AC,Astrazeneca,2021-01-17,13,12,1,0,0,0,<NA>,NaT,None,None
1,AC,Coronavac,2021-01-17,3978,3973,5,0,0,0,<NA>,NaT,None,None
2,AC,Pfizer,2021-01-17,4,3,0,1,0,0,<NA>,NaT,None,None
3,AC,Astrazeneca,2021-01-24,794,792,2,0,0,0,<NA>,NaT,None,None
4,AC,Coronavac,2021-01-24,5962,5951,11,0,0,0,<NA>,NaT,None,None


In [1]:
# df_vacina_proj['semana_proj'].max()
df_vacina_proj.iloc[20:30]

NameError: name 'df_vacina_proj' is not defined

## Query join com tabela de todas as combinações de vacina_apelido, sigla_uf e semana  (BigQuery)
### Dois problemas corrigidos: 1) essa tabela com todas as combinações e doses projetadas de 2020.

In [ ]:
df_vacina_proj = pd.io.gbq.read_gbq('''
SELECT COALESCE(wy.week, v.semana) as semana, COALESCE(wy.sigla_uf, v.sigla_uf) as sigla_uf, COALESCE(wy.vacina_apelido, v.vacina_apelido) as vacina_apelido, COALESCE(v.qt_total,wy.qt_total) as qt_total, COALESCE(v.qt_D1,wy.qt_D1) as qt_D1, COALESCE(v.qt_D2,wy.qt_D2) as qt_D2, COALESCE(v.qt_Reforco,wy.qt_Reforco) as qt_Reforco, COALESCE(v.qt_Adicional,wy.qt_Adicional) as qt_Adicional, COALESCE(v.qt_Unica,wy.qt_Unica) as qt_Unica, COALESCE(vp.qt_D2_Proj,0) as qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
FROM (
    WITH sundays AS (
  SELECT d AS week
  FROM UNNEST(GENERATE_DATE_ARRAY(DATE '2021-01-01', DATE '2022-12-31', INTERVAL 1 DAY)) d
  WHERE EXTRACT(DAYOFWEEK FROM d) = 1      -- Sundays
),
ufs AS (
  SELECT DISTINCT sigla_uf
  FROM `enapdatasets.vacinacao`
),
vacinas AS (
  SELECT vacina_apelido
  FROM UNNEST(['Coronavac','Pfizer','Janssen','Astrazeneca']) AS vacina_apelido
)
SELECT
  u.sigla_uf,  vi.vacina_apelido,  s.week,  0 AS qt_total,  0 AS qt_D1,  0 AS qt_D2,  0 AS qt_Reforco,  0 AS qt_Adicional,  0 AS qt_Unica
FROM ufs u
CROSS JOIN vacinas vi
CROSS JOIN sundays s
) wy
left join `enapdatasets.vacinacao` v on wy.week = v.semana and wy.vacina_apelido=v.vacina_apelido and wy.sigla_uf=v.sigla_uf
LEFT JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL @qtd_dias_proj_d2 DAY) as semana_proj
    FROM `enapdatasets.vacinacao` where EXTRACT(YEAR FROM semana) >= 2021
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=wy.week
order by v.sigla_uf, wy.week, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
''', project_id=project_id)

df_vacina_proj.head()

/tmp/ipython-input-244296362.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj = pd.io.gbq.read_gbq('''


,sigla_uf,vacina_apelido,semana,mes,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
0,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,1900-02-18,RO,Astrazeneca
1,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1900-02-18,RO,Coronavac
2,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1900-10-14,BA,Coronavac
3,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1901-02-17,BA,Pfizer
4,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1921-06-05,SC,Astrazeneca


### Código com a quanitdade de dias parametrizado
```
SELECT COALESCE(wy.week, v.semana) as semana, COALESCE(wy.sigla_uf, v.sigla_uf) as sigla_uf, COALESCE(wy.vacina_apelido, v.vacina_apelido) as vacina_apelido, COALESCE(v.qt_total,wy.qt_total) as qt_total, COALESCE(v.qt_D1,wy.qt_D1) as qt_D1, COALESCE(v.qt_D2,wy.qt_D2) as qt_D2, COALESCE(v.qt_Reforco,wy.qt_Reforco) as qt_Reforco, COALESCE(v.qt_Adicional,wy.qt_Adicional) as qt_Adicional, COALESCE(v.qt_Unica,wy.qt_Unica) as qt_Unica, COALESCE(vp.qt_D2_Proj,0) as qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
FROM (
    WITH sundays AS (
  SELECT d AS week
  FROM UNNEST(GENERATE_DATE_ARRAY(DATE '2021-01-01', DATE '2022-12-31', INTERVAL 1 DAY)) d
  WHERE EXTRACT(DAYOFWEEK FROM d) = 1      -- Sundays
),
ufs AS (
  SELECT DISTINCT sigla_uf
  FROM `enapdatasets.vacinacao`
),
vacinas AS (
  SELECT vacina_apelido
  FROM UNNEST(['Coronavac','Pfizer','Janssen','Astrazeneca']) AS vacina_apelido
)
SELECT
  u.sigla_uf,  vi.vacina_apelido,  s.week,  0 AS qt_total,  0 AS qt_D1,  0 AS qt_D2,  0 AS qt_Reforco,  0 AS qt_Adicional,  0 AS qt_Unica
FROM ufs u
CROSS JOIN vacinas vi
CROSS JOIN sundays s
) wy
left join `enapdatasets.vacinacao` v on wy.week = v.semana and wy.vacina_apelido=v.vacina_apelido and wy.sigla_uf=v.sigla_uf
LEFT JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL @qtd_dias_proj_d2 DAY) as semana_proj
    FROM `enapdatasets.vacinacao` where EXTRACT(YEAR FROM semana) >= 2021
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=wy.week
order by v.sigla_uf, wy.week, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj

```